In [1]:
!nvidia-smi

Mon Jul 17 16:08:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    48W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [1]:
from ultralytics import YOLO
import cv2

In [3]:
model = YOLO("/tmp/model/bestn.pt")

In [10]:
model=YOLO("/tmp/model/runs_0/train4/weights/best.pt")

In [2]:
import os
from google.cloud import storage

# Initialize Google Cloud Storage client
storage_client = storage.Client()
# Specify your bucket name and model folder
bucket_name = "trap-camera-data"
model_folder_name = "model"
# Define the temporary local directory path within the Vertex environment
temp_model_dir = "/tmp/model/"
# os.makedirs(temp_model_dir)
# Get the list of blobs (files) in the model folder
bucket = storage_client.bucket(bucket_name)
blobs = bucket.list_blobs(prefix=model_folder_name + "/")  # Specify the model folder as the prefix
# Copy each blob (file) from the model folder to the temporary local directory
for blob in blobs:
    if not blob.name.endswith('/'):  # Check if the blob is a file and not a directory
        # Construct the destination path for the file in the temporary directory
        relative_path = os.path.relpath(blob.name, model_folder_name)  # Get the relative path within the model folder
        destination_path = os.path.join(temp_model_dir, relative_path)

        # Create subdirectories if needed
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)

        # Download the file to the destination path
        blob.download_to_filename(destination_path)
        # print(f"Copied {blob.name} to {destination_path}")
# Now you can access the model files in the temporary local directory (e.g., /tmp/model)

In [8]:
model.train(data=r"/tmp/model/dataset_0/dataset.yaml", epochs=1000, project=r"/tmp/model/runs_0" )

Ultralytics YOLOv8.0.136 🚀 Python-3.10.11 torch-2.0.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16161MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/tmp/model/dataset_0/dataset.yaml, epochs=1000, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/tmp/model/runs_0, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optim

In [11]:
model.export(format='pb')
# model.export(format='torchscript')
# model.export(format='coreml')

Ultralytics YOLOv8.0.136 🚀 Python-3.10.11 torch-2.0.1+cu117 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients

PyTorch: starting from /tmp/model/runs_0/train4/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (5.9 MB)

TensorFlow SavedModel: starting export with tensorflow 2.11.0...

ONNX: starting export with onnx 1.14.0 opset 17...
ONNX: simplifying with onnxsim 0.4.33...


============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 2.7s, saved as /tmp/model/runs_0/train4/weights/best.onnx (11.6 MB)

TensorFlow SavedModel: running 'onnx2tf -i /tmp/model/runs_0/train4/weights/best.onnx -o /tmp/model/runs_0/train4/weights/best_saved_model -nuo --non_verbose'
TensorFlow SavedModel: export failure ❌ 13.7s: libusb-1.0.so.0: cannot open shared object file: No such file or directory

TensorFlow GraphDef: starting export with tensorflow 2.11.0...
TensorFlow GraphDef: export failure ❌ 0.0s: 'NoneType' object has no attribute 'inputs'


[]

In [12]:
import os
from google.cloud import storage
storage_client = storage.Client()
bucket_name = "trap-camera-data"
source_folder = "/tmp/model/runs_0/"
destination_folder = "model/runs_0"
files=[]
for root, _, filenames in os.walk(source_folder):
    for filename in filenames:
        files.append(os.path.join(root, filename))
bucket = storage_client.bucket(bucket_name)
for file_path in files:
    relative_path = os.path.relpath(file_path, source_folder)
    destination_path = os.path.join(destination_folder, relative_path)
    blob = bucket.blob(destination_path)
    blob.upload_from_filename(file_path)
    print(f"Transferred {file_path} to gs://{bucket_name}/{destination_path}")

Transferred /tmp/model/runs_0/train4/confusion_matrix_normalized.png to gs://trap-camera-data/model/runs_0/train4/confusion_matrix_normalized.png
Transferred /tmp/model/runs_0/train4/labels.jpg to gs://trap-camera-data/model/runs_0/train4/labels.jpg
Transferred /tmp/model/runs_0/train4/train_batch2.jpg to gs://trap-camera-data/model/runs_0/train4/train_batch2.jpg
Transferred /tmp/model/runs_0/train4/confusion_matrix.png to gs://trap-camera-data/model/runs_0/train4/confusion_matrix.png
Transferred /tmp/model/runs_0/train4/train_batch0.jpg to gs://trap-camera-data/model/runs_0/train4/train_batch0.jpg
Transferred /tmp/model/runs_0/train4/P_curve.png to gs://trap-camera-data/model/runs_0/train4/P_curve.png
Transferred /tmp/model/runs_0/train4/val_batch0_pred.jpg to gs://trap-camera-data/model/runs_0/train4/val_batch0_pred.jpg
Transferred /tmp/model/runs_0/train4/val_batch0_labels.jpg to gs://trap-camera-data/model/runs_0/train4/val_batch0_labels.jpg
Transferred /tmp/model/runs_0/train4/arg

In [1]:
from ultralytics import YOLO
model = YOLO('/tmp/model/runs_0/train4/weights/best.pt')

In [2]:
import cv2
result = model('tiger-species-sumatran-tigers-project-endangered-tigers.2.jpg')
result


image 1/1 /home/jupyter/tiger-species-sumatran-tigers-project-endangered-tigers.2.jpg: 640x576 1 Sumatran Tiger, 18.2ms
Speed: 6.9ms preprocess, 18.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 576)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'Asian Elephant', 1: 'Sun Bears', 2: 'Sumatran Tiger', 3: 'Pig Tailed Macaque', 4: 'Rhinoceros Hornbill'}
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        